# 0 - Setup

In [ ]:
!pip install --upgrade torchvision
!pip install --upgrade torch
import torch
import torchvision
print(f"torch version:{torch.__version__}")
print(f"torchvision version:{torchvision.__version__}")

In [ ]:
import matplotlib.pyplot as plt
from torch import nn
from torchvision import transforms

try:
  from torchinfo import summary
except:
  !pip install torchinfo
  from torchinfo import summary

In [ ]:
# Try to import the going_modular directory, download it from GitHub if it doesn't work
try:
    from going_modular.going_modular import data_setup, engine
    from helper_functions import download_data, set_seeds, plot_loss_curves
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular or helper_functions scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !mv pytorch-deep-learning/helper_functions.py . # get the helper_functions.py script
    !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine
    from helper_functions import download_data, set_seeds, plot_loss_curves

## Device Agnostic Code

In [ ]:
device="cuda" if torch.cuda.is_available() else "cpu"
device

# 1 - Getting Data
The dataset we're going to use for deploying a food101_mini_classification model is...

Pizza, steak, sushi 20% dataset (pizza, steak, sushi classes from Food101, random 20% of samples)

In [ ]:
# Download pizza, steak, sushi images from GitHub
data_20_percent_path=download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip",
                                     destination="pizza_steak_sushi_20_percent")
data_20_percent_path

In [ ]:
# setup training and test paths
train_dir=data_20_percent_path/"train"
test_dir=data_20_percent_path/"test"
train_dir,test_dir

# 2 - Creating an EffNetB2 feature extractor
Feautre extractor = a term for a transfer learning model that has its base layers frozen and output layers (or head layers) customized to a certain problem.

EffNetB2 pretrained model in PyTorch - https://pytorch.org/vision/stable/models/generated/torchvision.models.efficientnet_b2.html#torchvision.models.EfficientNet_B2_Weights

In [ ]:
import torchvision

# 1. setup pretrained EfficeintB2 weights
effnetb2_weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT #DEFAULT means best

# 2. Get EffNetB2 transforms
effnetb2_transform=effnetb2_weights.transforms()

# 3. Setup pretrained model instance
effnetb2 = torchvision.models.efficientnet_b2(weights=effnetb2_weights)

# 4. Freeze the base layers in the model (this will stop all layers from training)
for param in effnetb2.parameters():
  param.requires_grad=False



In [ ]:
from torchinfo import summary

# Print EffNetB2 model summary (uncomment for full output)
summary(effnetb2,
        input_size=(1, 3, 224, 224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

In [ ]:
effnetb2.classifier

In [ ]:
set_seeds()
effnetb2.classifier=nn.Sequential(nn.Dropout(p=0.3,inplace=True),
                                  nn.Linear(in_features=1408,out_features=3))

In [ ]:

from torchinfo import summary

# Print EffNetB2 model summary (uncomment for full output)
summary(effnetb2,
        input_size=(1, 3, 224, 224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

## 2.1 - Creating a function to make an EffNetB2 feature extractor

In [ ]:
def create_effnetb2_model(num_classes:int=3,
                          seed:int=42):
  weights=torchvision.models.EfficientNet_B2_Weights.DEFAULT
  transform=weights.transforms()
  model=torchvision.models.efficientnet_b2(weights=weights)



  for param in model.parameters():
    param.requires_grad=False
  torch.manual_seed(seed)
  model.classifier=nn.Sequential(nn.Dropout(p=0.3,inplace=True),
                                 nn.Linear(in_features=1408,out_features=num_classes))
  return model,transform


In [ ]:
effnetb2,effnetb2_transforms=create_effnetb2_model(num_classes=3,
                                                   seed=42)

In [ ]:
from torchinfo import summary
summary(model=effnetb2,
        input_size=(1,3,288,288),
        col_names=["input_size","output_size","num_params","trainable"],
        col_width=20,
        row_settings=["var_names"])

## 2.2 - Creating DataLoaders for EffNetB2

In [ ]:
from going_modular.going_modular import data_setup
train_dataloader_effnetb2,test_dataloader_effnetb2,class_names=data_setup.create_dataloaders(train_dir=train_dir,
                                                                                    test_dir=test_dir,
                                                                                    transform=effnetb2_transforms,
                                                                                    batch_size=32)


In [ ]:
len(train_dataloader_effnetb2),len(test_dataloader_effnetb2),class_names

## 2.3 - Training EffNetB2 feature extractor

In [ ]:
from going_modular.going_modular import engine

loss_fn=torch.nn.CrossEntropyLoss()

optimizer=torch.optim.Adam(params=effnetb2.parameters(),
                           lr=1e-3)#0.001

set_seeds()

effnetb2_results=engine.train(model=effnetb2,
                              train_dataloader=train_dataloader_effnetb2,
                              test_dataloader=test_dataloader_effnetb2,
                              epochs=10,
                              optimizer=optimizer,
                              loss_fn=loss_fn,
                              device=device)

## 2.4 - Inspecting EffNetB2 loss curves

In [ ]:
from helper_functions import plot_loss_curves

plot_loss_curves(effnetb2_results)

## 2.5 - Saving EffNetB2 feature extractor

In [ ]:
from going_modular.going_modular import utils

# Save the model
utils.save_model(model=effnetb2,
                 target_dir="models",
                 model_name="effnetb2_feature_extractor_food101_mini.pth")


## 2.6 - Inspecting the size of our EffNetB2 feature extractor

In [ ]:
from pathlib import Path

# Get the model size in bytes and convert to megabytes
pretrained_effnetb2_model_size = Path("models/effnetb2_feature_extractor_food101_mini.pth").stat().st_size / (1024 * 1024)
print(f"Pretrained EffNetB2 feature extractor model size: {round(pretrained_effnetb2_model_size, 2)} MB")

# 3 - Deployed Gradio app structure
Let's start to put all of our app files into a single directory:
```
Colab -> folder with all Gradio files -> upload app files to Hugging Face Spaces -> deploy
```
By the end our file structure will look like this:

```
demos/
└── food101_mini_classification/
    ├── effnetb2_feature_extractor_food101_mini.pth
    ├── app.py
    ├── examples/
    │   ├── example_1.jpg
    │   ├── example_2.jpg
    │   └── example_3.jpg
    ├── model.py
    └── requirements.txt
```


In [ ]:
import shutil
from pathlib import Path

# Create food101_mini_classification demo path
food101_mini_classification_demo_path=Path("demos/food101_mini_classification/")

# Remove files that might exist and create a new directory
if food101_mini_classification_demo_path.exists():
  shutil.rmtree(food101_mini_classification_demo_path)

# Create the new directory
food101_mini_classification_demo_path.mkdir(parents=True,
                                  exist_ok=True)

!ls demos/food101_mini_classification/


## 3.1 - Creating a folder of example images to use with our food101_mini_classification demo

What we want:

*    3 images in an examples/ directory
*    Images should be from the test set


In [ ]:
import shutil
from pathlib import Path

# Create an examples directory
food101_mini_classification_examples_path = food101_mini_classification_demo_path/"examples"
food101_mini_classification_examples_path.mkdir(parents=True,exist_ok=True)

# Collect three random test dataset image paths
food101_mini_classification_examples=[Path('data/pizza_steak_sushi_20_percent/test/pizza/482858.jpg'),
                          Path('data/pizza_steak_sushi_20_percent/test/pizza/3770514.jpg'),
                          Path('data/pizza_steak_sushi_20_percent/test/sushi/46797.jpg')]

# copy the three images to the example directory

for example in food101_mini_classification_examples:
  destination=food101_mini_classification_examples_path/example.name # .name returns the file name
  print(f"[INFO] Copying {example} to {destination}")
  shutil.copy2(src=example,
               dst=destination)



Let's now verify that we can get a list of lists from our `examples/` directory.

In [ ]:
import os
# Get example filepaths in a list of lists
example_list = [["examples/" + example] for example in os.listdir(food101_mini_classification_examples_path)]
example_list

## 3.2 - Moving our trained EffNetB2 model to our food101_mini_classification demo directory

In [ ]:
import shutil

# Create a source path for our target model
effnetb2_foodvision_mini_model_path = "models/09_pretrained_effnetb2_feature_extractor_pizza_steak_sushi_20_percent.pth"

# Create a destination path for our target model
effnetb2_foodvision_mini_model_destination = foodvision_mini_demo_path / effnetb2_foodvision_mini_model_path.split("/")[1]

# Try to move the model file
try:
  print(f"[INFO] Attempting to move {effnetb2_foodvision_mini_model_path} to {effnetb2_foodvision_mini_model_destination}")

  # Move the movel
  shutil.move(src=effnetb2_foodvision_mini_model_path,
              dst=effnetb2_foodvision_mini_model_destination)

  print(f"[INFO] Model move complete.")
# If the model has already been moved, check if it exists
except:
  print(f"[INFO] No model found at {effnetb2_foodvision_mini_model_path}, perhaps its already been moved?")
  print(f"[INFO] Model exists at {effnetb2_foodvision_mini_model_destination}: {effnetb2_foodvision_mini_model_destination.exists()}")


## 3.3 - Turning off EffNetB2 model into a Python script (model.py)

We have a saved `.pth` model `state_dict` and want to load it into a model instance.

Let's move our `create_effnetb2_model()` function to a script so we can reuse it.

In [ ]:
%%writefile demos/food101_mini_classification/model.py
import torch
import torchvision

from torch import nn

def create_effnetb2_model(num_classes:int=3,
                          seed:int=42):
  # 1, 2, 3 Create EffNetB2 pretrained weights, transforms and model
  weights=torchvision.models.EfficientNet_B2_Weights.DEFAULT
  transforms=weights.transforms()
  model=torchvision.models.efficientnet_b2(weights=weights)

  for param in model.parameters():
    param.requires_grad=False
    model.classifier=nn.Sequential(
        nn.Dropout(p=0.3,inplace=True),
        nn.Linear(in_features=1408,out_features=num_classes)
    )
  return model,transforms



## 3.4 - Turning our FoodVision Mini Gradio app into a Python script (`app.py`)

In [ ]:
%%writefile demos/food101_mini_classification/app.py
### 1. Imports and class names setup ###
import gradio as gr
import os
import torch

from model import create_effnetb2_model
from timeit import default_timer as timer
from typing import Tuple,Dict

# Setup class names
class_names=["pizza","steak","sushi"]
### 2. Model and transforms perparation ###
effnetb2,effnetb2_transforms=create_effnetb2_model(num_classes=len(class_names))

# Load save weights
effnetb2.load_state_dict(
    torch.load(
        f="09_pretrained_effnetb2_feature_extractor_pizza_steak_sushi_20_percent.pth",
        map_location=torch.device("cpu") # load the model to the CPU
    )
)

### 3. Predict function ###
def predict(img)->Tuple[Dict,float]:
  # start a timer
  start_time = timer()

  # Transform the input image for use with EffNetB2
  img=effnetb2_transforms(img).unsqueeze(0) # unsqueeze = add batch dimension on 0th index

  # Put model into eval mode, make prediction
  effnetb2.eval()
  with torch.inference_mode():
    # Pass transformed image through the model and turn the prediction logits into probaiblities
    pred_probs=torch.softmax(effnetb2(img),dim=1)

  # Create a prediction label and prediction probability dictionary
  pred_labels_and_probs = {}

  # pred_labels_and_probs = {class_names[i]: float(pred_probs[0][i]) for i in range(len(class_names))}
  # comment loop below to un-comment line above
  for i,class_name in enumerate(class_names):
    pred_labels_and_probs[class_name]=pred_probs[0][i]

  # Calculate pred time
  end_time=timer()
  pred_time=round(end_time-start_time,4)

  # Return pred dict and pred time
  return pred_labels_and_probs,pred_time

### 4. Gradio app ###

# Create title, description and article

title="FoodVision Mini"
description = "An [EfficientNetB2 feature extractor](https://pytorch.org/vision/stable/models/generated/torchvision.models.efficientnet_b2.html#torchvision.models.efficientnet_b2) computer vision model to classify images as pizza, steak or sushi."
article = "Created at [09. PyTorch Model Deployment](https://www.learnpytorch.io/09_pytorch_model_deployment/#74-building-a-gradio-interface)."

# Create example list
# Create example list
example_list = [["examples/" + example] for example in os.listdir("examples")]

# Create the Gradio demo
demo = gr.Interface(fn=predict, # maps inputs to outputs
                    inputs=gr.Image(type="pil"),
                    outputs=[gr.Label(num_top_classes=3, label="Predictions"),
                             gr.Number(label="Prediction time (s)")],
                    examples=example_list,
                    title=title,
                    description=description,
                    article=article)

# Launch the demo!
demo.launch()


## 3.5 - Creating a requirements file for FoodVision Mini (`requirements.txt`)

The requirements file will tell our Hugging Face Space what software dependencies our app requires.

The three main ones are:

*    `torch`
*    `torchvision`
*    `gradio`